<img src="entete-datathon.jpg" width=640/>

# Bienvenue dans l’aventure !
Prépare-toi à vivre **36 heures intenses de collaboration, d’analyse et d’innovation** où se rencontrent **finance, data science et intelligence artificielle.**

Le Datathon 2025, c’est bien plus qu’une compétition : c’est **une immersion dans la réalité de la finance,** où tes idées et ton esprit analytique feront la différence.

## Rappel du défi

**Contexte :** Dans un environnement financier marqué par une réglementation complexe et des sanctions économiques croissantes, votre défi est de transformer cette complexité réglementaire en opportunité d'aide à la décision pour la gestion de portefeuilles d'actions.

**Objectif :** Concevoir un outil d'analyse et de simulation utilisant l'IA générative pour analyser automatiquement les textes réglementaires, extraire les éléments clés, évaluer l'impact sur le S&P 500, et transformer ces résultats en recommandations stratégiques concrètes.

**Livrables attendus :**
- Simulation de différents scénarios
- Mesure des risques par entreprise, secteur ou zone géographique
- Propositions d'ajustements concrets (réallocation, rotation sectorielle, etc.)
- Outil flexible capable de traiter différents formats de documents réglementaires

**Il est fortement recommandé de lire attentivement l'énoncé complet dans le document DATATHON POLYFINANCES 2025 CHALLENGE qui a été partagé avec tous les participants pour comprendre tous les détails et critères d'évaluation.**

---

## Votre environnement SageMaker Unified Studio

Cet environnement SageMaker Unified Studio est mis à votre disposition pour le Datathon. Il vous offre un accès aux outils d'analyse et d'IA d'AWS dans un environnement Jupyter intégré.

Le fichier [getting_started.ipynb](getting_started.ipynb) contient une introduction aux fonctionnalités de SageMaker Unified Studio et vous guidera dans la prise en main de l'environnement.

**Important:** L'utilisation de cet environnement SageMaker n'est pas obligatoire. Vous disposez de droits d'administration sur votre compte AWS et êtes libres d'utiliser les services et outils de votre choix pour développer votre solution.



---

## Les données et documents de référence

Le jeu de données comprend plusieurs fichiers et dossiers contenant les informations nécessaires à la résolution du cas. Vous pouvez également utiliser des sources externes (Yahoo Finance, SEC, etc.).

**IMPORTANT** : Un nouveau document sera transmis dimanche matin pour l'évaluation finale.

**Contenu du jeu de données :**
- `2025-08-15_composition_sp500.csv` : Composition du S&P 500 (ticker, nom, poids, prix)
- `2025-09-26_stocks-performance.csv` : Métriques de performance (valorisation, EPS, FCF, etc.)
- **Dossier directives** : Documents législatifs et réglementaires (UE, US, Japon, Chine, Canada)
- **Dossier fillings** : Rapports 10-K 2024 par ticker (exposition réglementaire détaillée)


### Téléchargement des données


In [ ]:
import pandas as pd
import requests
import zipfile
import os
from tqdm import tqdm
from IPython.display import Markdown, display

In [ ]:
# Télécharger le fichier zip
url = 'https://d38aa0udflevgr.cloudfront.net/datathon/jeu_de_donnees.zip'
response = requests.get(url)
with open('jeu_de_donnees.zip', 'wb') as f:
    f.write(response.content)

# Extraire le fichier zip
with zipfile.ZipFile('jeu_de_donnees.zip', 'r') as zip_ref:
    for file in tqdm(zip_ref.infolist(), desc="Extraction"):
        zip_ref.extract(file, '.')

### Prévisualisation des données 

In [ ]:
%%local
# Composition du S&P 500
composition_sp500 = pd.read_csv("2025-08-15_composition_sp500.csv")
composition_sp500

In [ ]:
# Métriques de performance
stocks_performance = pd.read_csv('2025-09-26_stocks-performance.csv')
stocks_performance

## L'IA générative avec Amazon Bedrock

Exemple d'utilisation simple:

In [ ]:
import boto3

client = boto3.client("bedrock-runtime")

response = client.converse(
    modelId="global.anthropic.claude-sonnet-4-5-20250929-v1:0",
    messages=[
        {
            "role": "user",
            "content": [{"text": "Say hello in 3 different languages"}]
        }
    ],
    inferenceConfig={
        "temperature": 0.5,
    }
)

display(Markdown(response["output"]["message"]["content"][0]["text"]))


## Liste de tous les modèles de Bedrock

_À noter que pas tous les modèles sont disponible_

In [ ]:
# Create Bedrock client
bedrock = boto3.client('bedrock')

# List all foundation models
response = bedrock.list_foundation_models()

# Extract and display model IDs
model_ids = [model['modelId'] for model in response['modelSummaries']]

# Print all model IDs
for model_id in sorted(model_ids):
    print(model_id)

## Certains modèles sont uniquement disponibles via un profil d'inférence global

In [ ]:
import boto3

client = boto3.client('bedrock')

response = client.list_inference_profiles(typeEquals='SYSTEM_DEFINED')

# Filter for global profiles
global_profiles = [
    profile for profile in response['inferenceProfileSummaries']
    if profile['inferenceProfileId'].startswith('global.')
]

for profile in global_profiles:
    print(f"Global Profile ID: {profile['inferenceProfileId']}")



## Exemple d'extraction de données non-structurées en données structurées 

In [ ]:
%pip install "instructor[bedrock]"

In [ ]:
import boto3
import instructor
import os
from pydantic import BaseModel


bedrock_client = boto3.client('bedrock-runtime')
client = instructor.from_bedrock(bedrock_client)


class Company10K(BaseModel):
    company_name: str
    trading_symbol: str
    commission_file_no: str
    fiscal_year_end: str
    state_of_incorporation: str
    employer_id_no: str
    address: str
    phone_number: str
    exchange: str


company10k = client.chat.completions.create(
    modelId="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    messages=[
        {
            "role": "user", 
             "content": f"Extract: {open('fillings/AMZN/2025-02-07-10k-AMZN.html').read(64000)}" 
        },
    ],
    response_model=Company10K,
    inferenceConfig={
        "maxTokens": 64000,
    }
)

display(company10k)

---

## Interagir avec vos données grâce aux Agents

Les Agents sont une excellente façon d'interagir avec vos données de manière conversationnelle. Voici un exemple simple avec Strands Agent:

In [ ]:
%pip install strands-agents strands-agents-tools

In [ ]:
from strands import Agent
from strands_tools import file_read


# Create an agent with default settings
agent = Agent(tools=[file_read])

# Ask the agent a question
agent("Décrit les fichiers qui se trouve dans le dossier directives")
agent("Souhaite bonne chance aux participants du Datathon PolyFinances")

---

## À vous de jouer maintenant !

**Félicitations !** Vous avez maintenant les outils en main pour relever ce défi passionnant.

**Il est temps de créer votre propre notebook** et de laisser libre cours à votre créativité et votre expertise technique. 

**Bonne chance, et que le meilleur gagne !** 🚀

